In [11]:
#import seaborn as sns
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from tiny.tfidf import *
from tiny.usage import *

 
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from xgboost import XGBClassifier

from tiny.lda import *
from  tiny.util import *


#args = locals()
feature_label = get_stable_feature('0924')

train = feature_label[feature_label['sex'].notnull()]
test = feature_label[feature_label['sex'].isnull()]

X = train.drop(['sex', 'age', 'sex_age', 'device'], axis=1)[:10000]
Y = train['sex_age'][:10000]
Y_CAT = pd.Categorical(Y)


Y

2018-09-28 23:11:58,892 util_log.py[33] INFO Begin to run get_stable_feature with:['0924'], {}
2018-09-28 23:11:58,899 util_cache_file.py[79] DEBUG fn:get_stable_feature, para:['0924'], kw:{}
2018-09-28 23:11:58,901 util_cache_file.py[21] DEBUG try to read cache from file:./cache/get_stable_feature_['0924']_{}.csv, type:csv
2018-09-28 23:12:08,854 util_cache_file.py[35] DEBUG Return 72727 resut from file cache:./cache/get_stable_feature_['0924']_{}.csv
2018-09-28 23:12:08,858 util_log.py[38] INFO cost:   9.97 sec: ==='get_stable_feature' end (['0924'], {}) 


0        1-10
2        1-10
3         1-5
7         2-7
8        1-10
11        2-5
12        1-5
13        1-4
14        1-2
15        1-8
17        2-7
18        1-6
19        2-5
23        1-4
24        2-8
25        1-4
26        1-6
27        2-9
30        1-6
32        1-4
33        1-7
34        1-8
35        1-7
37        1-8
39        1-5
40        2-3
41        1-5
42        1-5
43        1-6
44        1-4
45        1-7
46        1-3
47        2-6
48        2-0
49        1-6
50        2-6
51        1-2
52        1-9
54        1-8
56        1-6
57        2-2
60        2-5
62        1-5
63       2-10
65        1-4
66        2-9
67        1-7
68        1-4
69        1-7
70        1-7
72        2-5
75        2-6
77        2-8
78        1-1
79        1-6
81        1-4
82        1-6
83        2-6
84        1-7
88        1-6
89        2-6
92        2-5
93        1-6
94        2-3
95       1-10
96        1-5
97        1-1
99        1-6
100       1-5
103       1-6
104      2-10
105   

In [13]:

X_train, X_test, y_train, y_test = train_test_split(X, Y_CAT.codes, test_size=0.3, random_state=666)




In [14]:
print( np.unique(y_train) )
print( np.unique(y_test) )

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


In [16]:

gbm = XGBClassifier(
                objective='multi:softprob',
                eval_metric='mlogloss',
                num_class=22,
                n_estimators=20,
                max_depth=3,

                min_child_weight=1,
                learning_rate=0.1,

                silent=True,
                gamma=0,
                max_delta_step=0,
                subsample=1,
                colsample_bytree=1,
                colsample_bylevel=1,
                reg_alpha=1,
                reg_lambda=1,
                scale_pos_weight=1,
                seed=1,
                missing=None)
# print(random_search.grid_scores_)
gbm.fit(X_train, y_train,  eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=True )

results = gbm.evals_result()

print(results)

[0]	validation_0-mlogloss:3.05177
Will train until validation_0-mlogloss hasn't improved in 50 rounds.
[1]	validation_0-mlogloss:3.01924
[2]	validation_0-mlogloss:2.99001
[3]	validation_0-mlogloss:2.96482
[4]	validation_0-mlogloss:2.94357
[5]	validation_0-mlogloss:2.92285
[6]	validation_0-mlogloss:2.90434
[7]	validation_0-mlogloss:2.88765
[8]	validation_0-mlogloss:2.87236
[9]	validation_0-mlogloss:2.8587
[10]	validation_0-mlogloss:2.84676
[11]	validation_0-mlogloss:2.8353
[12]	validation_0-mlogloss:2.82478
[13]	validation_0-mlogloss:2.81524
[14]	validation_0-mlogloss:2.80597
[15]	validation_0-mlogloss:2.79737
[16]	validation_0-mlogloss:2.79012
[17]	validation_0-mlogloss:2.78346
[18]	validation_0-mlogloss:2.77655
[19]	validation_0-mlogloss:2.77088
{'validation_0': {'mlogloss': [3.051766, 3.019241, 2.990009, 2.964824, 2.943568, 2.922853, 2.904342, 2.887646, 2.872357, 2.858696, 2.846756, 2.835304, 2.824777, 2.815245, 2.805973, 2.797369, 2.790115, 2.783456, 2.776547, 2.770876]}}


[3.051766,
 3.019241,
 2.990009,
 2.964824,
 2.943568,
 2.922853,
 2.904342,
 2.887646,
 2.872357,
 2.858696,
 2.846756,
 2.835304,
 2.824777,
 2.815245,
 2.805973,
 2.797369,
 2.790115,
 2.783456,
 2.776547,
 2.770876]

In [19]:
best_epoch = np.array(results['validation_0']['mlogloss']).argmin()+1
best_score = np.array(results['validation_0']['mlogloss']).min()

print(f'{best_epoch}_{best_score}')


20_2.770876
